In [ ]:
from flask import Flask,redirect,jsonify,render_template
from flask.ext.cors import CORS
app = Flask(__name__)
CORS(app)

# @app.route("/")
# def main():
# #     return render_template('index.html')
#     return "<a href='./news/111'>go</a>"
    
# @app.route("/news/")
# @app.route("/news/<id>")
# def news(id=None):
#     print(id)
#     if id is None:
# #         return "<a href='./news'>go</a>"
#         return redirect("/")
#     return "Hello"+id

@app.route("/api/count")
def count():
    cur.execute('SELECT date,count(timestamp) FROM cnynes GROUP BY date ORDER BY date DESC')
    data = []
    for x in cur.fetchall():
        temp = {}
        temp["date"] = x[0]
        temp["count"] = x[1]
        cur.execute('SELECT pic FROM cnynes WHERE pic is not null and date == "'+temp["date"]+'" LIMIT 1')
        pic = cur.fetchone()
        if pic is not None:
            temp["pic"] = pic[0]
        else:
            temp["pic"] = None
        
        data.append(temp)
    return jsonify(data)

@app.route("/api/date/<date>")
def read(date):
    print(date)
    cur.execute('SELECT *FROM cnynes WHERE date="'+date+'" ORDER BY timestamp DESC')
    data = []
    for x in cur.fetchall():
        print(x)
        temp = {}
#         temp["id"] = x[0]
#         temp["nid"] = x[1]
        temp["date"] = x[2]
        temp["timestamp"] = x[3]
        temp["url"] = x[4]
        temp["title"] = x[5]
        temp["content"] = x[6]
        temp["keyword"] = x[7]
#         temp["pic"] = x[8]
        
        data.append(temp)
    return jsonify(data)


if __name__ == "__main__":
    app.run(host='0.0.0.0')

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

## 連接和新建資料庫

In [ ]:
import sqlite3

def create_table(conn, create_table_sql):
    global cur
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        cur = conn.cursor()
        cur.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)
        
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None


db = "cnynes.sqlite"
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS cnynes (
                                        id integer PRIMARY KEY,
                                        nid integer NOT NULL,
                                        date text,
                                        timestamp text,
                                        url text,
                                        title text,
                                        content text,
                                        keyword text,
                                        pic text
                                    ); """

conn = create_connection(db)
if conn is not None:
    create_table(conn, sql_create_projects_table)
else:
    print("Error! cannot create the database connection.")

cur.execute('SELECT max(timestamp) FROM cnynes')
print("DB最後的新聞：",cur.fetchall()[0][0])

cur.execute('SELECT count(timestamp) FROM cnynes')
print("DB新聞比數：",cur.fetchall()[0][0])

cur.execute('SELECT count(timestamp) FROM cnynes WHERE keyword is null')
print("DB未萃取比數：",cur.fetchall()[0][0])

cur.execute('SELECT date,count(timestamp) FROM cnynes GROUP BY date ORDER BY date DESC')
print("每一天比數：",cur.fetchall())

# cur.execute('SELECT * FROM cnynes')
# print(cur.fetchall())

## 爬未載入db的新聞鏈接和標題

In [ ]:
import requests
import urllib.request 
# from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import json
import datetime

base_url = "https://news.cnyes.com"
url = base_url +"/api/v3/news/category/tw_quo?startAt=1512086400&endAt=1516406400&limit=30&page=1"

cur.execute('SELECT max(timestamp) FROM cnynes')
max_time = cur.fetchall()[0][0]
if max_time is None:
    max_time = 0
else:
    max_time = int(max_time)
print(max_time)
news = []
continueToUrl = True
while(True):
    print(url)
    r = requests.get(url)
    data = r.json()
    for item in data["items"]["data"]:
        new = {}
        new["timestamp"] = item["publishAt"]
        new["date"] = datetime.datetime.fromtimestamp(int(item["publishAt"])).strftime('%Y-%m-%d')
        new["nid"] = int(item["newsId"])
        new["url"] = "https://news.cnyes.com/news/id/"+str(item["newsId"])
        new["title"] = item["title"]
        
        if max_time < new["timestamp"]:
            print(new)
            news.append(new)
            colvalue = (new["timestamp"],new["date"],new["nid"],new["url"],new["title"])
            cur.execute("INSERT INTO cnynes('timestamp','date','nid','url','title') VALUES (?,?,?,?,?)",colvalue) 
            conn.commit()
            print(cur.fetchall())
        else:
            continueToUrl = False
            break
#         break
    
    if continueToUrl is True:
        if data["items"]["next_page_url"] is not None:
            url = base_url+data["items"]["next_page_url"]
            print("Continue to next page")
#         break
    else:
        break

## 爬未載入db的新聞內容

In [ ]:
cur.execute('SELECT id,url FROM cnynes WHERE content is null')
for id,url in cur.fetchall():
    print(id,url)
    r = requests.get(url)
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.content,"lxml")
    news_text = soup.select("div[itemprop:articlebody]")[1].text
    print(news_text)
    cur.execute("UPDATE cnynes SET content = ? WHERE id = ?",(news_text,id)) 
    conn.commit()  
    
    pic = soup.find("article").find('img')
    if pic is None:
        pic = None
    else:
        pic = pic.attrs['src']
        print(pic)
        cur.execute("UPDATE cnynes SET pic = ? WHERE id = ?",(pic,id)) 
        conn.commit()   
#     break

## 萃取未有keyword的資料

In [ ]:
import jieba
import jieba.analyse

def index_of_keyword(sentences, keyword):
    for i, s in enumerate(sentences):
        if keyword in s:
              return i
    return -1

cur.execute('SELECT id,content FROM cnynes WHERE keyword is null')
for id,content in cur.fetchall():
#     print(id,content)
    print(id)
    
    keywords = jieba.analyse.extract_tags(content, topK=3, withWeight=False)
    keywords2 = jieba.analyse.textrank(content, topK=3, withWeight=False) 
    keywords_all = keywords+keywords2

    s_index = []
    for keyword in keywords_all:
        sentences = content.replace("\n","").split(u"。")
        s_index.append(index_of_keyword(sentences,keyword))

    s_set = set(s_index)
    s_index = list(s_set)

    summary = ""
    for i in s_index:
        summary += sentences[i]+u"。\n"

#     print("content",content)
#     print ("summary",summary)
    
    conn.cursor().execute("UPDATE cnynes SET keyword = ? WHERE id = ?",(summary,id)) 
    conn.commit()        
#     break



## backup

In [ ]:
# cur.execute('SELECT id,url FROM cnynes WHERE pic is null')
# for id,url in cur.fetchall():
# #     print(id,url)
#     r = requests.get(url)
#     r.encoding = "utf-8"
#     soup = BeautifulSoup(r.content,"lxml")
#     pic = soup.find("article").find('img')
#     if pic is None:
#         pic = None
#     else:
#         pic = pic.attrs['src']
#         print(pic)
#         cur.execute("UPDATE cnynes SET pic = ? WHERE id = ?",(pic,id)) 
#         conn.commit()   
# #     break

In [ ]:
from bs4 import BeautifulSoup as soup

news_text = soup.find("article")
# if news_text is None:
#     news_text = None
# else:
#     news_text = news_text.attrs['src']
# print(news_text)

In [ ]:
.find('img')